# Vehicle Routing Problem

A Vehicle Routing Problem is a generalization of the TSP, so its part of the Routing Problems Family. The goal is to find the optimal set of routes (sequence of locations) for a fleet of vehicles. So our set of assumptions from TSP varies:


* There may be several routes (or vehicles) and capacity constraints. So there is a finite number of locations that a given vehicle may visit, depending of the demand of each visited node.
* A Depot is chosen.
* Cost from visting location j from location i may or may not be symmetric
* There may be or not some time window constraints. This a framework where a location can only be visited in a certain time window.
* The number of vehicles can be constrained or optimiced if fixed cost of adding a new vehicle is set.


## 1. Create distance Callback and input data as an list of (x,y) coordinates and a list of demand of each location

In [1]:
import math
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance(x1, y1, x2, y2):
    # Manhattan distance
    dist = abs(x1 - x2) + abs(y1 - y2)

    return dist

class CreateDistanceCallback(object):
    """Create callback to calculate distances between points."""

    def __init__(self, locations):
        """Initialize distance array."""
        size = len(locations)
        self.matrix = {}

        for from_node in range(size):
            self.matrix[from_node] = {}
            for to_node in range(size):
                x1 = locations[from_node][0]
                y1 = locations[from_node][1]
                x2 = locations[to_node][0]
                y2 = locations[to_node][1]
                self.matrix[from_node][to_node] = distance(x1, y1, x2, y2)

    def Distance(self, from_node, to_node):
        return int(self.matrix[from_node][to_node])

# Demand callback
class CreateDemandCallback(object):
    """Create callback to get demands at each location."""

    def __init__(self, demands):
        self.matrix = demands

    def Demand(self, from_node, to_node):
        return self.matrix[from_node]


## 2.Create data:

* A list of (x,y) coordinates for each location
* Demand for each location
* Set a location point as depot (each route must begin and end in that point)
* Set number of vehicles

In [2]:
def create_data_array():

    locations = [[82, 76], [96, 44], [50, 5], [49, 8], [13, 7], [29, 89], [58, 30], [84, 39],
                 [14, 24], [12, 39], [3, 82], [5, 10], [98, 52], [84, 25], [61, 59], [1, 65],
                 [88, 51], [91, 2], [19, 32], [93, 3], [50, 93], [98, 14], [5, 42], [42, 9],
                 [61, 62], [9, 97], [80, 55], [57, 69], [23, 15], [20, 70], [85, 60], [98, 5]]

    demands = [0, 19, 21, 6, 19, 7, 12, 16, 6, 16, 8, 14, 21, 16, 3, 22, 18,
               19, 1, 24, 8, 12, 4, 8, 24, 24, 2, 20, 15, 2, 14, 9]
    data = [locations, demands]
    return data

In [3]:
#data is returned a a list of two elements, a list of locations and a list of Demands
data = create_data_array()
locations = data[0]
demands = data[1]
num_locations = len(locations)
depot = 0  
num_vehicles = 5

In [4]:
#Get locations one and two coorindates:
data[0][0:2]

[[82, 76], [96, 44]]

In [5]:
#Get locations one and two demand:
data[1][0:2]

[0, 19]

## 3.Create Routing Model

In [6]:
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)
#search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

In [7]:
# Callback to the distance function.
dist_between_locations = CreateDistanceCallback(locations)
dist_callback = dist_between_locations.Distance
routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)

### 3.1 Create Capacity Dimension

**Dimensions:**

Routing problems involve quantities that accumulate along a vehicle's route. In this example, such quantity is demand — say, the weight or volume of a package that must be delivered. For every location where a vehicle stops along its route, the total demand on the vehicle increases by the demand at that location. (Other examples of these types of quantities are the distance a vehicle travels, or its travel time.)

The routing solver stores each quantity of this type in an object called a dimension. The dimension contains a callback for the quantity, along with related data and variables. You can add a dimension to a routing problem using the solver's AddDimension method

$$ \text{dimension: } D_{j}^{(i)} = \sum_{i=1}^{n_{j}} d_{i}$$
$$ \text{i: Locations}$$
$$ \text{j: vehicle}$$
$$ \text{nj: number of locations to be visited by vehicle j}$$

In [9]:
# Put a callback to the demands.
demands_at_locations = CreateDemandCallback(demands)
demands_callback = demands_at_locations.Demand

In [10]:
type(demands_at_locations)

__main__.CreateDemandCallback

In [11]:
type(demands_callback)

method

In [12]:
# Add a dimension for demand.
slack_max = 0
#Set max capacity for each vehicle
vehicle_capacity = 100
fix_start_cumul_to_zero = True
demand = "Demand"
routing.AddDimension(demands_callback, slack_max, vehicle_capacity,
                     fix_start_cumul_to_zero, demand)

True

### 3.2 Solver

In [13]:
# Solve, displays a solution if any.
assignment = routing.SolveWithParameters(search_parameters)
if assignment:
    # Display solution.
    # Solution cost.
    print("Total distance of all routes: " + str(assignment.ObjectiveValue()) + "\n") 

    for vehicle_nbr in range(num_vehicles):
        index = routing.Start(vehicle_nbr)
        index_next = assignment.Value(routing.NextVar(index))
        route = ''
        route_dist = 0
        route_demand = 0

        while not routing.IsEnd(index_next):
            node_index = routing.IndexToNode(index)
            node_index_next = routing.IndexToNode(index_next)
            route += str(node_index) + " -> "
            # Add the distance to the next node.
            route_dist += dist_callback(node_index, node_index_next)
            # Add demand.
            route_demand += demands[node_index_next]
            index = index_next
            index_next = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        node_index_next = routing.IndexToNode(index_next)
        route += str(node_index) + " -> " + str(node_index_next)
        route_dist += dist_callback(node_index, node_index_next)
        print("Route for vehicle " + str(vehicle_nbr) + ":\n\n" + route + "\n") 
        print("Distance of route " + str(vehicle_nbr) + ": " + str(route_dist)) 
        print( "Demand met by vehicle " + str(vehicle_nbr) + ": " + str(route_demand) + "\n")

Total distance of all routes: 970

Route for vehicle 0:

0 -> 14 -> 2 -> 3 -> 23 -> 4 -> 11 -> 28 -> 6 -> 26 -> 0

Distance of route 0: 300
Demand met by vehicle 0: 100

Route for vehicle 1:

0 -> 27 -> 24 -> 0

Distance of route 1: 78
Demand met by vehicle 1: 44

Route for vehicle 2:

0 -> 20 -> 5 -> 25 -> 10 -> 15 -> 22 -> 9 -> 8 -> 18 -> 29 -> 0

Distance of route 2: 316
Demand met by vehicle 2: 98

Route for vehicle 3:

0 -> 12 -> 1 -> 16 -> 30 -> 0

Distance of route 3: 96
Demand met by vehicle 3: 72

Route for vehicle 4:

0 -> 13 -> 21 -> 31 -> 19 -> 17 -> 7 -> 0

Distance of route 4: 180
Demand met by vehicle 4: 96

